In [42]:
import numpy as np
import sys
import pandas as pd
import redis
import pickle
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp

In [43]:
class FuelType(models.Model):
    name = models.CharField(max_length=50)
    is_active = models.BooleanField(default=False)
    is_secondary = models.BooleanField(default=False)

    class Meta:
        db_table = 'fuel_types'

    def __unicode__():
        return '%s - %s' % (id, name)

NameError: name 'models' is not defined

In [38]:
TOP_ORDER_LISTINGS_STATUSES = ['active', 'refurbishing']

def pandas_cached_Aquery(query, disable_cache=True):
    return pd.read_sql_query(query)
    
LISTING_FEATURE_QUERY = """SELECT listings.id, localities.city_id, IFNULL(price, 0) < 200000 AS price_b_2,
                           IFNULL(price, 0) BETWEEN 100000 AND 300000 AS price_bw_1_3, IFNULL(price, 0) BETWEEN 200000
                           AND 400000 AS price_bw_2_4, IFNULL(price, 0) BETWEEN 300000 AND 500000 AS price_bw_3_5,
                           IFNULL(price, 0) BETWEEN 400000 AND 600000 AS price_bw_4_6, IFNULL(price, 0) BETWEEN 500000
                           AND 700000 AS price_bw_5_7, IFNULL(price, 0) BETWEEN 600000 AND 800000 AS price_bw_6_8,
                           IFNULL(price, 0) BETWEEN 700000 AND 900000 AS price_bw_7_9, IFNULL(price, 0) > 900000 AS
                           price_a_9, IFNULL(listings.body_type_id = 1, 0) AS hatchback, IFNULL(listings.body_type_id = 
                           2, 0) AS sedan, IFNULL(listings.body_type_id = 3, 0) AS mpv, IFNULL(listings.body_type_id = 
                           4, 0) AS suv, listings.fuel_type_id_primary = 1 AS petrol, listings.fuel_type_id_primary = 2 
                           AS diesel, IFNULL(owners, 0) = 1 AS first_owner, IFNULL(owners, 0) > 1 AS owner, 
                           IFNULL(mileage, 0) <= 30000 AS mileage_b_3, IFNULL(mileage, 0) BETWEEN 30001 AND 70000 AS 
                           mileage_bw_3_7, IFNULL(mileage, 0) > 70000 AS mileage_a_7, IFNULL(year(curdate())-
                           year(registration_date), 0) <= 2 AS age_b_2, IFNULL(year(curdate())-year(registration_date), 
                           0) BETWEEN 3 AND 4 AS age_bw_3_4, IFNULL (year(curdate())- year(registration_date), 0) 
                           BETWEEN 5 AND 7 AS age_bw_5_7, IFNULL(year(curdate())- year(registration_date), 0)>7 AS 
                           age_a_7,  IFNULL(variants.model_id, 0) IN (SELECT variants.model_id FROM listings JOIN 
                           variants ON listings.variant_id=variants.id WHERE listings.id IN ({listing_ids})) AS model_id,
                           IFNULL(variants.id, 0) IN (SELECT variants.id FROM listings JOIN variants ON listings.
                           variant_id=variants.id WHERE listings.id IN ({listing_ids})) AS variant_id, IFNULL(listings.
                           color_id, 0) IN (1 , 3, 6) AS color_wsg, IFNULL(listings.color_id, 0) IN (2 , 13) AS color_bo,
                           IFNULL(listings.color_id, 0) NOT IN (1 , 2, 3, 6, 13) AS color_rest, IFNULL(CAST(listings.
                           is_inventory AS UNSIGNED), 0) AS is_inventory, price, created_at FROM listings JOIN 
                           localities ON listings.locality_id = localities.id JOIN variants ON listings.variant_id = 
                           variants.id where {filter_query}"""

In [39]:
def get_listing_features(city_id, listings=None, inactive=False, listing_filter=True):
    city_filter = """localities.city_id = {city_id}""".format(city_id=city_id)
    listing_id_filter = """listings.id in ({listings})""".format(
        listings=','.join(map(str, list(listings)))
    ) if listings and listing_filter else None
    listing_status_filter = """listings.status in {status}""".format(
        status=tuple(TOP_ORDER_LISTINGS_STATUSES)
    ) if not inactive else None

    filter_array = [city_filter, listing_id_filter, listing_status_filter]

    listings_features = pandas_cached_query(
        LISTING_FEATURE_QUERY.format(
            filter_query=' and '.join(filter(None, filter_array)),
            listing_ids=','.join(map(str, listings)) if listings else 0
        )
    )

In [40]:
def get_similar_cars_from_filters(input_filters):
    city_active_listings = np.load('/home/pavank/ds/docker/api/backend/city_active_listings.npy')
    city_active_listings_features = np.array(city_active_listings)[:, 2:17]
    city_active_listings_ids = np.array(city_active_listings)[:, 0]

    int_max = sys.maxint
    fields = ["price_b_2", "price_bw_1_3", "price_bw_2_4", "price_bw_3_5", "price_bw_4_6", "price_bw_5_7",
              "price_bw_6_8", "price_bw_7_9", "price_a_9", "Hatchback", "Sedan", "MPV", "SUV", "Petrol", "Diesel"]
    final = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype='int32')

    price_slab_field_mapping = {
        "price_b_2": {'min': 0, 'max': 2000000},
        "price_bw_1_3": {'min': 100000, 'max': 300000},
        "price_bw_2_4": {'min': 200000, 'max': 400000},
        "price_bw_3_5": {'min': 300000, 'max': 500000},
        "price_bw_4_6": {'min': 400000, 'max': 600000},
        "price_bw_5_7": {'min': 500000, 'max': 700000},
        "price_bw_6_8": {'min': 600000, 'max': 800000},
        "price_bw_7_9": {'min': 700000, 'max': 900000},
        "price_a_9": {'min': 900000, 'max': int_max},
    }
    body_types = dict()

    for body_type_obj in list(BodyType.objects.values('id', 'name')):
        body_types[str(body_type_obj['id'])] = body_type_obj['name']

    fuel_types = dict()

    for fuel_type_obj in list(FuelType.objects.filter(is_active=True, is_secondary=False).values('id', 'name')):
        fuel_types[str(fuel_type_obj['id'])] = fuel_type_obj['name']

    if 'price_min' in input_filters.keys():
        price_min = input_filters['price_min']
    else:
        price_min = 0

    if 'price_max' in input_filters.keys():
        price_max = input_filters['price_max']
    else:
        price_max = int_max - 1

    for key, value in price_slab_field_mapping.iteritems():
        if int(price_min) >= value['min'] and int(price_max) < value['max']:
            final[fields.index(key)] = 1

    if 'body_type' in input_filters.keys():
        body_type = input_filters['body_type']
        body_type = body_type.split(',')
        for key in body_type:
            if key in list(range(1, 5)):
                final[fields.index(body_types[key])] = 1

    if 'fuel_type' in input_filters.keys():
        fuel_type = input_filters['fuel_type']
        fuel_type = fuel_type.split(',')
        for key in fuel_type:
            final[fields.index(fuel_types[key])] = 1

    listing_car_features = np.asarray([final])

    similarity_indices = np.argsort(
        -cosine_similarity(city_active_listings_features, listing_car_features).reshape(1, -1))

    similar_listings = city_active_listings_ids[similarity_indices].flatten()

    return similar_listings

In [41]:
input_filters = {'sort': 'r', 'price_max': 6500000, 'cars_seen': '84360,86280', 'years': '2015-01-01',
                         'user_search': 'true', 'model': '689', 'price_min': '700000', 'slug': 'used-cars-in-mumbai'}
get_similar_cars_from_filters(input_filters)

NameError: global name 'BodyType' is not defined